## Import the libraries

In [56]:
import pandas as pd
import pickle
import time

from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import leaguegamefinder


In [4]:
season = '2021-22'
season_type = 'Regular Season'

## Move the data to the script folder

### Loading the historical data (do not run it)

In [3]:
# import pickle

# path_game_by_game = 'G:/My Drive/Colab Notebooks/00 - Lambton/2022.1/04 - AML3406 - AI and ML Capstone Project/GitHub/Player-Performance-Improvement-Prediction/Data Gathering/Datasets/df_2018_to_2022_with_next_game_draftkings_score.pkl'
# path_mean_10_games = 'G:/My Drive/Colab Notebooks/00 - Lambton/2022.1/04 - AML3406 - AI and ML Capstone Project/GitHub/Player-Performance-Improvement-Prediction/Data Gathering/Datasets/mean_10_games_df.pkl'

# with open(path_game_by_game, 'rb') as f:
#     game_by_game_df = pickle.load(f)

# with open(path_mean_10_games, 'rb') as f:
#     mean_10_games_df = pickle.load(f)

In [4]:
# mean_10_games_df.columns

In [5]:
# columns_to_drop = ['Opp - Minutes played', 'Opp - Field Goals Made', 
#        'Opp - Field Goals Attempted', 'Opp - Field Goals 3 Points Made', 
#        'Opp - Field Goal 3 Points Attempt', 'Opp - Free Throws Made', 
#        'Opp - Free Throws Attempt', 'Opp - Offensive Rebounds', 
#        'Opp - Defensive Rebounds', 'Opp - Assists',
#        'Opp - Steals', 'Opp - Blocks', 'Opp - Turnovers',
#        'Opp - Personal Fouls', 'Opp - Points',
#        'Opp - Player Efficiency Ratings', 'Opp - Double-double',
#        'Opp - Triple-double']

# mean_10_games_df.drop(columns=columns_to_drop, inplace=True, axis=1)
# mean_10_games_df.columns

### Moving the datasets to the local folder (do not run it)

In [6]:
# reference_path_game_by_game = 'G:/My Drive/Colab Notebooks/00 - Lambton/2022.1/04 - AML3406 - AI and ML Capstone Project/GitHub/Player-Performance-Improvement-Prediction/Scripts/game_by_game_dataset.pkl'
# reference_path_mean_10_games = 'G:/My Drive/Colab Notebooks/00 - Lambton/2022.1/04 - AML3406 - AI and ML Capstone Project/GitHub/Player-Performance-Improvement-Prediction/Scripts/mean_10_games_dataset.pkl'

# with open(reference_path_game_by_game, 'wb') as f:
#     pickle.dump(game_by_game_df, f)

# with open(reference_path_mean_10_games, 'wb') as f:
#     pickle.dump(mean_10_games_df, f)

## Load the historical data

In [1]:
import pickle

reference_path_game_by_game = 'G:/My Drive/Colab Notebooks/00 - Lambton/2022.1/04 - AML3406 - AI and ML Capstone Project/GitHub/Player-Performance-Improvement-Prediction/Scripts/game_by_game_dataset.pkl'
reference_path_mean_10_games = 'G:/My Drive/Colab Notebooks/00 - Lambton/2022.1/04 - AML3406 - AI and ML Capstone Project/GitHub/Player-Performance-Improvement-Prediction/Scripts/mean_10_games_dataset.pkl'

with open(reference_path_game_by_game, 'rb') as f:
    game_by_game_df = pickle.load(f)

with open(reference_path_mean_10_games, 'rb') as f:
    mean_10_games_df = pickle.load(f)

In [8]:
game_by_game_df.head(2)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Blocks,Turnovers,Personal Fouls,Points,Player Efficiency Ratings,Double-double,Triple-double,DraftKings score,Draftkings next game score,Next Opponent
0,22018,101108,Chris Paul,0021800009,2018-10-17,HOU vs. NOP,HOU,NOP,1,0,...,0,1,3,19,33.838,0,0,62.25,68.25,LAL
1,22018,101108,Chris Paul,0021800035,2018-10-20,HOU @ LAL,HOU,LAL,0,1,...,0,3,2,28,49.812,1,0,68.25,65.00,LAC


In [9]:
mean_10_games_df.head(2)

,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,...,Player Efficiency Ratings,Double-double,Triple-double,Game Date,Season ID,Player ID,Player Name,Game ID,Draftkings score,Draftkings next game score
0,31.7,7.6,15.5,1.5,5.4,2.3,3.0,0.6,4.5,1.6,...,26.9807,0.0,0.0,2018-11-04,22018,1628398,Kyle Kuzma,0021800139,28.75,27.50
1,35.0,9.6,19.1,1.5,5.2,6.1,7.9,0.8,6.8,7.7,...,39.8832,0.5,0.1,2018-11-04,22018,2544,LeBron James,0021800139,22.00,43.25


In [10]:
mean_10_games_df.columns

Index(['Minutes played', 'Field Goals Made', 'Field Goals Attempted',
       'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt',
       'Free Throws Made', 'Free Throws Attempt', 'Offensive Rebounds',
       'Defensive Rebounds', 'Assists', 'Steals', 'Blocks', 'Turnovers',
       'Personal Fouls', 'Points', 'Player Efficiency Ratings',
       'Double-double', 'Triple-double', 'Game Date', 'Season ID', 'Player ID',
       'Player Name', 'Game ID', 'Draftkings score',
       'Draftkings next game score'],
      dtype='object')

In [9]:
reference_date = mean_10_games_df['Game Date'].max()
reference_date

Timestamp('2022-03-23 00:00:00')

In [48]:
active_players = players.get_active_players()
players_ids = [str(player['id']) for player in active_players]

### Get data from the current season

In [13]:
columns = ['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE']

latest_request = pd.DataFrame(columns=columns)

In [14]:
for idx, id in enumerate(players_ids):
    try:
        data = playergamelog.PlayerGameLog(
            player_id=id,
            season=season,
            season_type_all_star=season_type,
            league_id_nullable='00',
            timeout=10)
    
        latest_request = pd.concat([latest_request, data.get_data_frames()[0]])
        
        # Preventing timeout exceptions
        time.sleep(.600)
    except:
        print(f'There was some problem while gathering data.\nPlayer id: {id}\nIteration #{idx}\n')

latest_request.head(3)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22021,1628398,0022101050,"MAR 18, 2022",WAS @ NYK,L,35,6,17,0.353,...,4,5,9,0,1,4,5,18,3,1
1,22021,1628398,0022101035,"MAR 16, 2022",WAS vs. DEN,L,27,3,10,0.3,...,8,9,7,1,0,2,1,6,-22,1
2,22021,1628398,0022101025,"MAR 14, 2022",WAS @ GSW,L,31,4,12,0.333,...,7,8,3,0,0,3,3,11,-16,1


In [5]:
latest_request['GAME_DATE'].iloc[0]

'MAR 18, 2022'

In [6]:
type(latest_request['GAME_DATE'].iloc[0])

str

In [7]:
latest_request['GAME_DATE'] =  pd.to_datetime(latest_request['GAME_DATE'])
type(latest_request['GAME_DATE'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [103]:
new_df = latest_request[latest_request['GAME_DATE'] > reference_date].copy()
new_df.head(2)

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22021,203484,0022101101,2022-03-25,WAS @ DET,W,33,2,10,0.2,...,2,3,2,1,0,1,2,8,-2,1
1,22021,203484,0022101097,2022-03-24,WAS @ MIL,L,33,5,16,0.313,...,4,5,2,2,0,1,2,16,-2,1


In [104]:
new_df.shape

(294, 27)

## Adjust the columns names

In [105]:
new_column_names = ['Season ID', 'Player ID', 'Game ID', 'Game Date', 'Matchup', 'Won or Lost',
       'Minutes played', 'Field Goals Made', 'Field Goals Attempted', 'Field Goals %', 
       'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt', 'Field Goal 3 Points %', 
       'Free Throws Made', 'Free Throws Attempt', 'Free Throw %', 'Offensive Rebounds', 'Defensive Rebounds', 'Rebounds', 
       'Assists', 'Steals', 'Blocks', 'Turnovers', 'Personal Fouls',
       'Points', 'Plus Minus', 'Video Available']
    
new_df.columns = new_column_names

## Drop the irrelevant features

In [106]:
columns_to_drop = ['Field Goals %', 'Field Goal 3 Points %', 
                'Free Throw %', 'Rebounds', 'Plus Minus', 'Video Available']

new_df.drop(columns=columns_to_drop, inplace=True, axis=1)

In [ ]:
new_df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won or Lost,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,22021,203484,0022101101,2022-03-25,WAS @ DET,W,33,2,10,0,...,4,4,1,2,2,1,0,1,2,8
1,22021,203484,0022101097,2022-03-24,WAS @ MIL,L,33,5,16,3,...,3,4,1,4,2,2,0,1,2,16


## Checking the missing values

In [ ]:
new_df.isna().sum().sum()

32

In [107]:
new_df.dropna(inplace=True)
new_df.isna().sum().sum()

0

## Check for duplicated rows

In [108]:
new_df.duplicated().sum()

0

## Reset the index

In [109]:
new_df.reset_index(drop=True, inplace=True)

## Adjust the data types

In [110]:
new_df['Won or Lost'].replace({'W': 1, 'L': 0}, inplace=True)
new_df.rename(columns={'Won or Lost': 'Won'}, inplace=True)
new_df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,22021,203484,0022101101,2022-03-25,WAS @ DET,1,33,2,10,0,...,4,4,1,2,2,1,0,1,2,8
1,22021,203484,0022101097,2022-03-24,WAS @ MIL,0,33,5,16,3,...,3,4,1,4,2,2,0,1,2,16


### Adjust the numeric features types

In [111]:
new_df.dtypes

Season ID                              object
Player ID                              object
Game ID                                object
Game Date                      datetime64[ns]
Matchup                                object
Won                                     int64
Minutes played                         object
Field Goals Made                       object
Field Goals Attempted                  object
Field Goals 3 Points Made              object
Field Goal 3 Points Attempt            object
Free Throws Made                       object
Free Throws Attempt                    object
Offensive Rebounds                     object
Defensive Rebounds                     object
Assists                                object
Steals                                 object
Blocks                                 object
Turnovers                              object
Personal Fouls                         object
Points                                 object
dtype: object

In [112]:
columns_to_int = ['Minutes played', 'Field Goals Made', 'Field Goals Attempted',
       'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt',
       'Free Throws Made', 'Free Throws Attempt', 'Offensive Rebounds',
       'Defensive Rebounds', 'Assists', 'Steals', 'Blocks', 'Turnovers',
       'Personal Fouls', 'Points']

columns_to_str = ['Season ID', 'Player ID', 'Game ID', 'Matchup']

In [113]:
new_df[columns_to_int] = new_df[columns_to_int].apply(pd.to_numeric)
new_df[columns_to_str] = new_df[columns_to_str].apply(lambda x: x.astype(str))
new_df.dtypes

Season ID                              object
Player ID                              object
Game ID                                object
Game Date                      datetime64[ns]
Matchup                                object
Won                                     int64
Minutes played                          int64
Field Goals Made                        int64
Field Goals Attempted                   int64
Field Goals 3 Points Made               int64
Field Goal 3 Points Attempt             int64
Free Throws Made                        int64
Free Throws Attempt                     int64
Offensive Rebounds                      int64
Defensive Rebounds                      int64
Assists                                 int64
Steals                                  int64
Blocks                                  int64
Turnovers                               int64
Personal Fouls                          int64
Points                                  int64
dtype: object

In [114]:
new_df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,22021,203484,0022101101,2022-03-25,WAS @ DET,1,33,2,10,0,...,4,4,1,2,2,1,0,1,2,8
1,22021,203484,0022101097,2022-03-24,WAS @ MIL,0,33,5,16,3,...,3,4,1,4,2,2,0,1,2,16


## Include aditional features

### Include player name

In [115]:
active_players[0:2]

[{'id': 1630173,
  'full_name': 'Precious Achiuwa',
  'first_name': 'Precious',
  'last_name': 'Achiuwa',
  'is_active': True},
 {'id': 203500,
  'full_name': 'Steven Adams',
  'first_name': 'Steven',
  'last_name': 'Adams',
  'is_active': True}]

In [116]:
active_players[players_ids.index('203500')]['full_name']

'Steven Adams'

In [117]:
new_df['Player Name'] = new_df['Player ID'].apply(lambda x: str(active_players[players_ids.index(str(x))]['full_name']))
new_df.head(2)

,Season ID,Player ID,Game ID,Game Date,Matchup,Won,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,...,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,Player Name
0,22021,203484,0022101101,2022-03-25,WAS @ DET,1,33,2,10,0,...,4,1,2,2,1,0,1,2,8,Kentavious Caldwell-Pope
1,22021,203484,0022101097,2022-03-24,WAS @ MIL,0,33,5,16,3,...,4,1,4,2,2,0,1,2,16,Kentavious Caldwell-Pope


### Include the players team name

In [118]:
game_finder = leaguegamefinder.LeagueGameFinder(season_nullable=season, season_type_nullable=season_type)
games = game_finder.get_data_frames()[0]
games.head(2)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1612709910,FWN,Fort Wayne Mad Ants,2022100461,2022-03-27,FWN @ MNE,None,36,18,...,1.0,1,8,9,6,4,1,3,2,-4.0
1,22021,1612709915,MNE,Maine Celtics,2022100461,2022-03-27,MNE vs. FWN,None,59,22,...,NaN,4,10,14,8,2,1,4,3,4.0


In [119]:
def get_team_name(player_id, game_id):
    if new_df[(new_df['Player ID'] == player_id) & (new_df['Game ID'] == game_id)]['Won'].values[0]:
        return games[(games['GAME_ID'] == game_id) & (games['WL'] == 'W')]['TEAM_ABBREVIATION'].values[0]
    else:
        return games[(games['GAME_ID'] == game_id) & (games['WL'] == 'L')]['TEAM_ABBREVIATION'].values[0]

In [120]:
player_team = []

for idx, row in new_df.iterrows():
    player_team.append(get_team_name(row['Player ID'], row['Game ID']))

new_df['Player Team'] = player_team

### Include Home x Away

In [121]:
new_df['Home'] = new_df['Matchup'].apply(lambda x: 0 if x[4] == '@' else 1)

### Include opponent

In [ ]:
new_df['Opponent'] = new_df['Matchup'].apply(lambda x: x[-3:])
new_df.head()

### Rearrange columns order

In [123]:
new_columns_order = ['Season ID', 'Player ID', 'Player Name',
                    'Game ID', 'Game Date', 'Matchup', 'Player Team', 'Opponent', 'Home', 'Won',
                    'Minutes played', 'Field Goals Made', 'Field Goals Attempted',
                    'Field Goals 3 Points Made', 'Field Goal 3 Points Attempt',
                    'Free Throws Made', 'Free Throws Attempt', 'Offensive Rebounds',
                    'Defensive Rebounds', 'Assists', 'Steals', 'Blocks', 'Turnovers',
                    'Personal Fouls', 'Points']

In [124]:
new_df = new_df[new_columns_order]
new_df.head(2)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,22021,203484,Kentavious Caldwell-Pope,0022101101,2022-03-25,WAS @ DET,WAS,DET,0,1,...,4,4,1,2,2,1,0,1,2,8
1,22021,203484,Kentavious Caldwell-Pope,0022101097,2022-03-24,WAS @ MIL,WAS,MIL,0,0,...,3,4,1,4,2,2,0,1,2,16


### Include PER (Player Efficiency Ratings)

#### PER = (FGM x 85.910 + Steals x 53.897 + 3PTM x 51.757 + FTM x 46.845 + Blocks x 39.190 + Offensive_Reb x 39.190 + Assists x 34.677 + Defensive_Reb x 14.707 — Foul x 17.174 — FT_Miss x 20.091 — FG_Miss x 39.190 — TO x 53.897) x (1 / Minutes)

Reference: https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20

In [125]:
pers = []
for idx, row in new_df.iterrows():
    if row['Minutes played'] == 0:
        per = 0
    else:
        per = round((row['Field Goals Made'] * 85.910 + row['Steals'] * 53.897 + row['Field Goals 3 Points Made'] * 51.757 + row['Free Throws Made'] * 46.845 + row['Blocks'] * 39.190 + row['Offensive Rebounds'] * 39.190 + row['Assists'] * 34.677 + row['Defensive Rebounds'] * 14.707 - row['Personal Fouls'] * 17.174 - (row['Free Throws Attempt'] - row['Free Throws Made']) * 20.091 - (row['Free Throws Attempt'] - row['Free Throws Made']) * 39.190 - row['Turnovers'] * 53.897) * (1 / row['Minutes played']), 3)
    pers.append(per)

new_df['Player Efficiency Ratings'] = pers

In [126]:
new_df.head(2)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,Player Efficiency Ratings
0,22021,203484,Kentavious Caldwell-Pope,0022101101,2022-03-25,WAS @ DET,WAS,DET,0,1,...,4,1,2,2,1,0,1,2,8,14.025
1,22021,203484,Kentavious Caldwell-Pope,0022101097,2022-03-24,WAS @ MIL,WAS,MIL,0,0,...,4,1,4,2,2,0,1,2,16,25.848


### Include double-double and triple-double features

DraftKings criteria:
- Double-double (max 1 per player): 
    - Points, Rebounds, Assists, Blocks, Steals
- Triple-double (max 1 per player): 
    - Points, Rebounds, Assists, Blocks, Steals

In [127]:
def get_double_and_triple_doubles(player_info):
    points = player_info['Points']
    rebounds = player_info['Offensive Rebounds'] + player_info['Defensive Rebounds']
    assists = player_info['Assists']
    blocks = player_info['Blocks']
    steals = player_info['Steals']

    stats = [points, rebounds, assists, blocks, steals]

    cnt = 0
    for stat in stats:
        if int(stat) >= 10:
            cnt += 1

    if cnt >= 3:
        return 'Triple-double'
    elif cnt == 2:
        return 'Double-double'
    else: 
        return None

In [128]:
new_df = new_df.reset_index(drop=True)

results = []

for index, row in new_df.iterrows():
    results.append(get_double_and_triple_doubles(row))

new_df['Double/Triple double'] = results

### Create the double-double and triple-double features

In [129]:
new_df['Double-double'] = new_df['Double/Triple double'].apply(lambda x: 1 if x == 'Double-double' else 0)
new_df['Triple-double'] = new_df['Double/Triple double'].apply(lambda x: 1 if x == 'Triple-double' else 0)

new_df.head(3)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,Player Efficiency Ratings,Double/Triple double,Double-double,Triple-double
0,22021,203484,Kentavious Caldwell-Pope,0022101101,2022-03-25,WAS @ DET,WAS,DET,0,1,...,2,1,0,1,2,8,14.025,None,0,0
1,22021,203484,Kentavious Caldwell-Pope,0022101097,2022-03-24,WAS @ MIL,WAS,MIL,0,0,...,2,2,0,1,2,16,25.848,None,0,0
2,22021,201580,JaVale McGee,0022101099,2022-03-24,PHX @ DEN,PHX,DEN,0,1,...,0,0,0,0,3,3,3.666,None,0,0


In [130]:
new_df.drop(['Double/Triple double'], axis=1, inplace=True)
new_df.head(2)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Defensive Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,Player Efficiency Ratings,Double-double,Triple-double
0,22021,203484,Kentavious Caldwell-Pope,0022101101,2022-03-25,WAS @ DET,WAS,DET,0,1,...,2,2,1,0,1,2,8,14.025,0,0
1,22021,203484,Kentavious Caldwell-Pope,0022101097,2022-03-24,WAS @ MIL,WAS,MIL,0,0,...,4,2,2,0,1,2,16,25.848,0,0


### Include DraftKings score (model's output)

In [131]:
def get_draftkings_score(player_info):
    points = player_info['Points']
    three_points = player_info['Field Goals 3 Points Made']
    rebounds = player_info['Offensive Rebounds'] + player_info['Defensive Rebounds']
    assists = player_info['Assists']
    steals = player_info['Steals']
    blocks = player_info['Blocks']
    turnovers = player_info['Turnovers']
    double_double = player_info['Double-double']
    triple_double = player_info['Triple-double']

    return points*1 + three_points*0.5 + rebounds*1.25 + assists*1.5 * steals*2 + blocks*2 + turnovers*(-0.5) + double_double*1.5 + triple_double*3

In [132]:
new_df = new_df.reset_index(drop=True)

results = []

for index, row in new_df.iterrows():
    results.append(get_draftkings_score(row))

new_df['DraftKings score'] = results
new_df.head(3)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points,Player Efficiency Ratings,Double-double,Triple-double,DraftKings score
0,22021,203484,Kentavious Caldwell-Pope,0022101101,2022-03-25,WAS @ DET,WAS,DET,0,1,...,2,1,0,1,2,8,14.025,0,0,17.25
1,22021,203484,Kentavious Caldwell-Pope,0022101097,2022-03-24,WAS @ MIL,WAS,MIL,0,0,...,2,2,0,1,2,16,25.848,0,0,35.25
2,22021,201580,JaVale McGee,0022101099,2022-03-24,PHX @ DEN,PHX,DEN,0,1,...,0,0,0,0,3,3,3.666,0,0,4.25


### Include the next Draftking score (not in use)

In [ ]:
# new_df = new_df.sort_values(['Player ID', 'Game Date'], ascending=True)
# new_df.head()

In [134]:
# new_df.reset_index(drop=True, inplace=True)
# draftkings_score_next_game = []

# for index, row in new_df.iterrows():
#     if index + 1 == new_df.shape[0]:
#         draftkings_score_next_game.append(None)
#     else: 
#         if new_df['Player ID'].iloc[index + 1] == row['Player ID']:
#             draftkings_score_next_game.append(new_df['DraftKings score'].iloc[index + 1])
#         else:
#             draftkings_score_next_game.append(None)

In [135]:
# new_df['Draftkings next game score'] = draftkings_score_next_game

### Include the next opponent (not in use)

In [136]:
# new_df.reset_index(drop=True, inplace=True)
# next_opponent = []

# for index, row in new_df.iterrows():
#     if index + 1 == new_df.shape[0]:
#         next_opponent.append(None)
#     else: 
#         if new_df['Player ID'].iloc[index + 1] == row['Player ID']:
#             next_opponent.append(new_df['Opponent'].iloc[index + 1])
#         else:
#             next_opponent.append(None)

In [137]:
# new_df['Next Opponent'] = next_opponent

In [ ]:
# new_df.isna().sum()

**There is no need to drop the na values, since these values wont be used to predict the new values. They only make sense for training the model!**

## Join new and historical data

In [139]:
game_by_game_df.sort_values('Game Date', ascending=False).head(3)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Blocks,Turnovers,Personal Fouls,Points,Player Efficiency Ratings,Double-double,Triple-double,DraftKings score,Draftkings next game score,Next Opponent
80346,22021,204001,Kristaps Porzingis,0022101097,2022-03-24,WAS @ MIL,WAS,MIL,0,0,...,0,2,5,16,27.740,0,0,45.25,66.50,DET
80337,22021,1626195,Willy Hernangomez,0022101098,2022-03-24,NOP vs. CHI,NOP,CHI,1,1,...,0,0,0,6,23.285,0,0,8.50,7.00,SAS
80329,22021,1630530,Trey Murphy III,0022101098,2022-03-24,NOP vs. CHI,NOP,CHI,1,1,...,0,0,1,7,28.955,0,0,10.00,4.75,SAS


In [140]:
new_df.sort_values('Game Date', ascending=False).head(3)

,Season ID,Player ID,Player Name,Game ID,Game Date,Matchup,Player Team,Opponent,Home,Won,...,Blocks,Turnovers,Personal Fouls,Points,Player Efficiency Ratings,Double-double,Triple-double,DraftKings score,Draftkings next game score,Next Opponent
188,22021,1630631,Jose Alvarado,0022101107,2022-03-26,NOP vs. SAS,NOP,SAS,1,0,...,1,0,1,23,45.941,0,0,47.25,NaN,None
175,22021,1630563,Joshua Primo,0022101107,2022-03-26,SAS @ NOP,SAS,NOP,0,1,...,1,3,1,12,10.115,0,0,14.25,NaN,None
29,22021,1627749,Dejounte Murray,0022101107,2022-03-26,SAS @ NOP,SAS,NOP,0,1,...,0,2,3,15,31.293,0,1,147.75,NaN,None


### Update the historical dataset

In [145]:
game_by_game_df = pd.concat([game_by_game_df, new_df])
game_by_game_df.sort_values('Game Date', inplace=True)
game_by_game_df.reset_index(drop=True, inplace=True)

In [155]:
with open(reference_path_game_by_game, 'wb') as f:
    pickle.dump(game_by_game_df, f)

## Prepare the 'mean' dataset

In [193]:
def get_player_mean_stats(new_df, game_by_game_df, n_games):
    
    columns_to_drop = ['Season ID', 'Player ID', 'Player Name', 
                    'Game ID', 'Game Date', 'Matchup', 'Player Team', 
                    'Opponent', 'Home', 'Won', 'DraftKings score']
    
    players_ids = new_df['Player ID'].unique()

    mean_values = []
    game_dates = []
    season_ids = []
    player_ids_list = []
    player_names = []
    game_ids = []
    draftkings_scores = []
    draftkings_scores_next_game = []
    
    for player_id in players_ids:
        number_of_new_rows = new_df[new_df['Player ID'] == player_id].shape[0]
        sub_df = game_by_game_df[game_by_game_df['Player ID'] == player_id].sort_values('Game Date', ascending=False).head(number_of_new_rows + n_games)
        sub_df = sub_df.sort_values('Game Date', ascending=True)
        sub_df.reset_index(drop=True, inplace=True)
        for index, row in sub_df.iterrows():
            if index + 1 > n_games:
                mean_values.append(sub_df.iloc[index+1-n_games : index+1].drop(columns_to_drop, axis=1).mean())
                game_dates.append(row['Game Date'])
                season_ids.append(row['Season ID'])
                player_ids_list.append(row['Player ID'])
                player_names.append(row['Player Name'])
                game_ids.append(row['Game ID'])
                draftkings_scores.append(row['DraftKings score'])
                                
    results_mean = pd.DataFrame(data=mean_values)
    results_mean['Game Date'] = game_dates
    results_mean['Season ID'] = season_ids
    results_mean['Player ID'] = player_ids_list
    results_mean['Player Name'] = player_names
    results_mean['Game ID'] = game_ids
    results_mean['Draftkings score'] = draftkings_scores    

    return results_mean

In [194]:
new_mean_10_games_df = get_player_mean_stats(new_df, game_by_game_df, 10)

,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,...,Player Efficiency Ratings,Double-double,Triple-double,Game Date,Season ID,Player ID,Player Name,Game ID,Draftkings score,Draftkings next game score
0,33.2,5.7,11.8,0.9,3.5,3.6,3.9,0.0,4.0,12.5,...,33.8155,0.8,0.0,2022-03-24,22021,101108,Chris Paul,0022101099,63.5,NaN


In [196]:
mean_10_games_df.sort_values('Game Date', ascending=False).head(3)

,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,...,Player Efficiency Ratings,Double-double,Triple-double,Game Date,Season ID,Player ID,Player Name,Game ID,Draftkings score,Draftkings next game score
65639,21.9,4.9,10.3,2.3,5.2,1.5,2.0,0.0,1.7,1.9,...,27.0958,0.0,0.0,2022-03-23,22021,1630613,Duane Washington Jr.,0022101086,11.75,2.5
65628,34.9,10.4,20.1,3.0,7.3,4.7,5.3,0.2,4.4,6.5,...,41.4631,0.1,0.0,2022-03-23,22021,1626164,Devin Booker,0022101090,52.25,147.5
65618,27.3,3.9,10.0,2.1,6.5,1.1,1.5,0.5,4.7,2.8,...,24.3502,0.0,0.0,2022-03-23,22021,203109,Jae Crowder,0022101090,26.00,25.5


In [197]:
new_mean_10_games_df.sort_values('Game Date', ascending=False).head(3)

,Minutes played,Field Goals Made,Field Goals Attempted,Field Goals 3 Points Made,Field Goal 3 Points Attempt,Free Throws Made,Free Throws Attempt,Offensive Rebounds,Defensive Rebounds,Assists,...,Player Efficiency Ratings,Double-double,Triple-double,Game Date,Season ID,Player ID,Player Name,Game ID,Draftkings score,Draftkings next game score
188,23.8,4.1,9.1,1.0,3.2,1.2,1.7,1.1,2.1,5.0,...,28.8571,0.1,0.0,2022-03-26,22021,1630631,Jose Alvarado,0022101107,47.25,NaN
175,24.7,2.4,6.1,0.7,3.2,1.0,1.4,1.1,2.9,1.3,...,13.8235,0.0,0.0,2022-03-26,22021,1630563,Joshua Primo,0022101107,14.25,NaN
29,35.9,8.7,20.2,1.5,5.2,5.0,6.0,0.7,8.1,9.1,...,38.3333,0.6,0.1,2022-03-26,22021,1627749,Dejounte Murray,0022101107,147.75,NaN


In [198]:
new_mean_10_games_df.shape

(262, 25)

In [ ]:
new_mean_10_games_df.isnull().sum()

### Update the historical dataset

In [201]:
mean_10_games_df = pd.concat([mean_10_games_df, new_mean_10_games_df])
mean_10_games_df.sort_values('Game Date', inplace=True)
mean_10_games_df.reset_index(drop=True, inplace=True)

In [208]:
with open(reference_path_mean_10_games, 'wb') as f:
    pickle.dump(mean_10_games_df, f)